In [ ]:
# BRESSOT Arnaud 11505990
# COMAS Valentin 11500223

In [2]:
# Import

import gzip # pour décompresser les données
import pickle 
import matplotlib.pyplot as plt # pour l'affichage
import torch,torch.utils.data
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
# Recuperation des données

# nombre d'image lues à chaque fois dans la base d'apprentissage (laisser à 1 sauf pour la question optionnelle sur les minibatchs)
TRAIN_BATCH_SIZE = 1
# on charge les données de la base MNIST
data = pickle.load(gzip.open('mnist.pkl.gz'),encoding='latin1')
# images de la base d'apprentissage
train_data = data[0][0]
# labels de la base d'apprentissage
train_data_label = data[0][1]
# images de la base de test
test_data = data[1][0]
# labels de la base de test
test_data_label = data[1][1]

In [19]:
# Partie 1 : Perceptron

def test_perceptron(tests):
    print("Weight - Nb Step")
    for i in test :
        weight = i[0]
        nb_step = i[1]
        w = np.full((785, 10), weight)

        for d in range(len(train_data)):
            x = train_data[d]
            x = np.append(x, 1)
            y = np.dot(x, w)
            dl = train_data_label[d]
            a = nb_step * (dl - y)
            b = np.outer(x, a)
            w += b

        nbCorrect = 0
        for d in range(len(test_data)):
            x = test_data[d]
            x = np.append(x, 1)
            y = np.dot(x, w)
            index = np.argmax(test_data_label[d])
            index2 = np.argmax(y)
            if index == index2:
                nbCorrect += 1
        print(i[0],"-",i[1]," : ",nbCorrect/len(test_data) * 100,"%")
    


test = [(0.5,0.001), (0.7,0.001), (0.3,0.001), (-0.5 ,0.001), (-0.7 ,0.001), (-0.3 ,0.001), (0.5,0.01), (0.5,0.1), (0.5,0.005), (0.5,0.0001), (0.5,0.00001)]
test_perceptron(test)

# On remarque que le poids initial n'est pas déterminant sur le résulat mais que c'est surtout le pas qui est important.

Weight - Nb Step
0.5 - 0.001  :  83.67142857142858 %
0.7 - 0.001  :  83.67142857142858 %
0.3 - 0.001  :  83.67142857142858 %
-0.5 - 0.001  :  83.67142857142858 %
-0.7 - 0.001  :  83.67142857142858 %
-0.3 - 0.001  :  83.67142857142858 %
0.5 - 0.01  :  75.55714285714285 %
0.5 - 0.1  :  9.785714285714285 %
0.5 - 0.005  :  80.58571428571429 %
0.5 - 0.0001  :  84.82857142857144 %
0.5 - 1e-05  :  79.98571428571428 %


In [38]:
# Partie 2 : Shallow network

nb_hn = 150
nb_step = 0.001

def shallow_network(nb_hn, nb_step, weight):
    w_h = np.full((np.shape(train_data)[1] + 1, nb_hn), weight)
    w_o = np.random.randn(nb_hn, np.shape(train_data_label)[1])
    for i in range(5):
        for d in range(len(train_data)):
            # Choisir entrée
            x = train_data[d]
            x = np.append(x, 1.)

            # Propagation yi(1)
            y_h = 1. / (1. + np.exp(-np.dot(x, w_h)))

            # Propagation yi(2)
            y_o = np.dot(y_h, w_o)

            # Rétro Propagation 

            # Calcul de l'erreur
            dl = train_data_label[d]
            m_o = dl - y_o

            # Rétro Propa de l'erreur
            a1 = y_h * (1 - y_h)
            a2 = np.dot(w_o, m_o)
            m_h = a1 * a2

            # Modification poids
            dw_o = nb_step * np.outer(y_h, m_o)
            w_o += dw_o
            dw_h = nb_step * np.outer(x, m_h)
            w_h += dw_h
            
    nbCorrect = 0
    for d in range(len(test_data)):
        x = test_data[d]
        x = np.append(x, 1)

        y_h = 1. / (1. + np.exp(-np.dot(x, w_h)))

        y_o = np.dot(y_h, w_o)
        index = np.argmax(test_data_label[d])
        index2 = np.argmax(y_o)
        if index == index2:
            nbCorrect += 1
    return nbCorrect/len(test_data) * 100

def test_shallow_network(tests):
    print("Nb de neuronnes cachés - Taille du pas - Poids initial couche cachée")
    for i in tests:
        result = shallow_network(i[0],i[1],i[2])
        print(i[0],"-",i[1],"-",i[2]," : ",result,"%")
    
tests = [(150, 0.001, 0.),(100, 0.001, 0.),(50, 0.001, 0.),(200, 0.001, 0.),(150, 0.01, 0.),(150, 0.1, 0.),(150, 0.0001, 0.),(150, 0.00001, 0.), (150, 0.001, 0.5),(150, 0.001, 0.8),(150, 0.001, -0.5),(150, 0.001, -0.8)]
test_shallow_network(tests)


Nb de neuronnes cachés - Taille du pas - Poids initial couche cachée
150 - 0.001 - 0.0  :  94.0 %
100 - 0.001 - 0.0  :  93.8 %
50 - 0.001 - 0.0  :  92.9 %
200 - 0.001 - 0.0  :  94.38571428571429 %
150 - 0.01 - 0.0  :  95.02857142857142 %
150 - 0.1 - 0.0  :  58.94285714285714 %
150 - 0.0001 - 0.0  :  83.81428571428572 %
150 - 1e-05 - 0.0  :  81.24285714285713 %
150 - 0.001 - 0.5  :  9.785714285714285 %
150 - 0.001 - 0.8  :  9.785714285714285 %
150 - 0.001 - -0.5  :  9.685714285714287 %
150 - 0.001 - -0.8  :  9.785714285714285 %


In [45]:
#  Partie 3 : Deep network

# Import as Tensors

TRAIN_BATCH_SIZE = 64

data = pickle.load(gzip.open('mnist.pkl.gz'),encoding='latin1')
train_data = torch.Tensor(data[0][0])
train_data_label = torch.Tensor(data[0][1])
test_data = torch.Tensor(data[1][0])
test_data_label = torch.Tensor(data[1][1])
train_dataset = torch.utils.data.TensorDataset(train_data,train_data_label)
test_dataset = torch.utils.data.TensorDataset(test_data,test_data_label)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

class DeepNN(nn.Module):
    def __init__(self, layers_size, activation_function):
        super(DeepNN, self).__init__()
        self.activation_function = activation_function
        self.layers = nn.ModuleList()
        #create the layers
        for i in range(len(layers_size) - 1):
            layer = nn.Linear(layers_size[i], layers_size[i+1])
            self.layers.append(layer)

    def forward(self, x):
        for l in self.layers:
            x = self.activation_function(l(x))
        return x

def instance_DeepNN(nb_step, epochs, layers_size, activation_function):
    deepNN = DeepNN(layers_size, activation_function)
    optimizer = torch.optim.Adam(deepNN.parameters(), lr=nb_step)

    for epoch in range(epochs):
        deepNN.train()
        for batch_ep, (image,label) in enumerate(train_loader):
            optimizer.zero_grad()
            output = deepNN(image)
            loss_f = nn.MSELoss()
            loss = loss_f(output, label)
            loss.backward()
            optimizer.step()

    deepNN.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for image, label in test_loader:
            output = deepNN(image)
            lose_f = nn.MSELoss()
            test_loss += lose_f(output, label).item() 
            pred = output.argmax(dim=1, keepdim=True)
            correct += int(np.argmax(output)) == int(np.argmax(label))

    test_loss /= len(test_loader.dataset)
    
    return (test_loss, 100. * correct / len(test_loader.dataset))
            
def test_DeepNN(tests):
    print("Nb de pas - Epochs - Layers_size - Fonction")
    for i in tests:
        test_loss, percent = instance_DeepNN(i[0], i[1], i[2], i[3])
        print(i[0],"-",i[1],"-",i[2],"-",i[3]," : Average loss : ",test_loss," Accuracy : ",percent,"%")

tests = [(0.001, 6, [784, 150 ,30, 10], nn.Sigmoid()), (0.01, 6, [784, 150 ,30, 10], nn.Sigmoid()), (0.0001, 6, [784, 150 ,30, 10], nn.Sigmoid()), (0.001, 4, [784, 150 ,30, 10], nn.Sigmoid()), (0.001, 8, [784, 150 ,30, 10], nn.Sigmoid()), (0.001, 6, [784, 150 ,30, 10], nn.Sigmoid()), (0.001, 6, [784, 10], nn.Sigmoid()),(0.001, 6, [784, 150 ,100, 50, 10], nn.Sigmoid()),(0.001, 6, [784, 300, 150 ,30, 10], nn.Sigmoid()), (0.001, 6, [784, 150 ,30, 10], nn.ReLU()), (0.01, 6, [784, 150 ,30, 10], nn.ReLU()), (0.0001, 6, [784, 150 ,30, 10], nn.ReLU()), (0.001, 4, [784, 150 ,30, 10], nn.ReLU()), (0.001, 8, [784, 150 ,30, 10], nn.ReLU()), (0.001, 6, [784, 150 ,30, 10], nn.ReLU()), (0.001, 6, [784, 10], nn.ReLU()),(0.001, 6, [784, 300, 150 ,30, 10], nn.ReLU())]


test_DeepNN(tests)

# On trouve le meilleur résulat avec 0.001 - 8 - [784, 150, 30, 10] - Sigmoid().


Nb de pas - Epochs - Layers_size - Fonction
0.001 - 6 - [784, 150, 30, 10] - Sigmoid()  : Average loss :  0.005562063564721549  Accuracy :  96.65714285714286 %
0.01 - 6 - [784, 150, 30, 10] - Sigmoid()  : Average loss :  0.006332226827426504  Accuracy :  96.24285714285715 %
0.0001 - 6 - [784, 150, 30, 10] - Sigmoid()  : Average loss :  0.050461277777861274  Accuracy :  69.75714285714285 %
0.001 - 4 - [784, 150, 30, 10] - Sigmoid()  : Average loss :  0.0075352249823294444  Accuracy :  95.6 %
0.001 - 8 - [784, 150, 30, 10] - Sigmoid()  : Average loss :  0.0046503896221581405  Accuracy :  97.07142857142857 %
0.001 - 6 - [784, 150, 30, 10] - Sigmoid()  : Average loss :  0.005772379419083986  Accuracy :  96.5 %
0.001 - 6 - [784, 10] - Sigmoid()  : Average loss :  0.016524196397277625  Accuracy :  91.6 %
0.001 - 6 - [784, 150, 100, 50, 10] - Sigmoid()  : Average loss :  0.005506017547457824  Accuracy :  96.38571428571429 %
0.001 - 6 - [784, 300, 150, 30, 10] - Sigmoid()  : Average loss :  0.

In [47]:
# Partie 4 : Pour aller plus loin

# another load to allow transformations

import torchvision

train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('Mnist', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor()
                             ])),
  batch_size=TRAIN_BATCH_SIZE, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('Mnist', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor()
                             ])),
  batch_size=1, shuffle=True)
class Conv2DNN(nn.Module):
    def __init__(self, activation_function):
        super(Conv2DNN, self).__init__()
        self.activation_function = activation_function
        
        #create the layers with a dropout of 0.25 for each convolutionnal layer to prevent overfitting
        self.drop = nn.Dropout2d(0.25)
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.lin = nn.Linear(64 * 5 * 5, 64)
        self.lin2 = nn.Linear(64, 10)
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.max_pool(x)
        x = self.activation_function(x)
        x = self.conv2(x)
        x = self.drop(x)
        x = self.max_pool(x)
        x = self.activation_function(x)
        
        x = x.view(x.size(0), -1)

        x = self.lin(x)
        x = self.activation_function(x)
        x = self.lin2(x)
        return x
            
nb_step = 0.001
epochs = 5
activation_function = nn.Sigmoid()

log = True

conv2dNN = Conv2DNN(activation_function)
optimizer = torch.optim.Adam(conv2dNN.parameters(), lr=nb_step)

for epoch in range(epochs):
    conv2dNN.train()
    for batch_ep, (image,label) in enumerate(train_loader):
        optimizer.zero_grad()
        output = conv2dNN(image)
        loss_f = nn.CrossEntropyLoss()
        loss = loss_f(output, label)
        loss.backward()
        optimizer.step()

conv2dNN.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for image, label in test_loader:
        output = conv2dNN(image)
        lose_f = nn.CrossEntropyLoss()
        test_loss += lose_f(output, label).item() 
        pred = output.argmax(dim=1, keepdim=True)
        correct += np.argmax(output) == label[0]

    test_loss /= len(test_loader)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
# Après plusieurs tests, nous en avons conclu que ce sont les meilleurs parametres pour ce réseau.


Test set: Average loss: 0.0777, Accuracy: 9749/10000 (97%)

